In [1]:
# Doing by file until I can figure out OAuth
import os
token_file = os.path.join(os.path.expanduser("~/"), ".jira_auth")
with open(token_file, 'r') as fd:
    uname = fd.readline().strip()  # Can't hurt to be paranoid
    pwd = fd.readline().strip()

SERVER = "https://jira.lsstcorp.org/"
from jira import JIRA
js = JIRA(server=SERVER, basic_auth=(uname, pwd))

In [2]:
query = "project = Simulations AND issuetype = Epic AND summary ~ \"SOCS Release\" ORDER BY key"
issues = js.search_issues(query)
print len(issues)

12


In [3]:
def get_linked_epics(issue, linkTypeName="Containment"):
    linked_epics = []
    links = issue.fields.issuelinks
    for link in links:
        if link.type.name == linkTypeName:
            linked_epics.append(js.issue(link.outwardIssue.key))
    return linked_epics

In [4]:
def get_component(istr):
    return istr.split()[0]

In [5]:
def create_list_from_numbered_description(desrc, bulletType="*"):
    import re
    olist = []
    for line in desrc.strip().split(os.linesep):
        values = line.split()
        if re.match("\d+\.\d+", values[0]) is None:
            numbered = bulletType
        else:
            if USE_MARKDOWN:
                numbered = " " * 2 + bulletType
            else:
                numbered = bulletType * 2
        olist.append("{0} {1}".format(numbered, " ".join(values[1:])))
    return olist

In [6]:
USE_MARKDOWN = False
# Write some boiler plate for the page
page_content = []
import time
page_content.append("This page provides the coordinated work plan between the Simulated OCS (SOCS) and the Scheduler.")
if USE_MARKDOWN:
    page_content.append("")
page_content.append("Updated: {0}".format(time.strftime("%Y-%m-%d %H:%M", time.localtime())))
page_content.append("")

In [7]:
for issue in issues:
    socs_summary = issue.fields.summary
    version = socs_summary.split()[-1]
    if USE_MARKDOWN:
        page_content.append("## Combined Release {0}".format(version))
    else:
        page_content.append("h2. Combined Release {0}".format(version))
    #print issue.fields.summary
    socs_descr = issue.fields.description
    comb_duedate = issue.fields.duedate
    if USE_MARKDOWN:
        page_content.append("**Release Date:** {0}".format(comb_duedate))
    else:
        page_content.append("*Release Date:* {0}".format(comb_duedate))
    page_content.append("")
    #sub_epics = get_linked_epics(issue)

    if USE_MARKDOWN:
        socs_work = create_list_from_numbered_description(socs_descr, "1.")
    else:
        socs_work = create_list_from_numbered_description(socs_descr)
    
    # Should only be one!
    sched_epic = get_linked_epics(issue, "Relates")[0]
    sched_summary = sched_epic.fields.summary
    sched_descr = sched_epic.fields.description
    if USE_MARKDOWN:
        sched_work = create_list_from_numbered_description(sched_descr, "1.")
    else:
        sched_work = create_list_from_numbered_description(sched_descr)
    
    # Create Table 
    if USE_MARKDOWN:
        page_content.append("{0} Workplan | {1} Workplan".format(socs_summary.split()[0], sched_summary.split()[0]))
        page_content.append("--- | ---")
    else:
        page_content.append("|| {0} Workplan || {1} Workplan ||".format(socs_summary.split()[0], sched_summary.split()[0]))
    
    if not USE_MARKDOWN:
        page_content.append("| {0} | {1} |".format(os.linesep.join(socs_work), os.linesep.join(sched_work)))
    else:    
        import itertools
        for ssw, sdw in itertools.izip_longest(socs_work, sched_work, fillvalue=" "):
            page_content.append("{0} | {1}".format(ssw, sdw))
                        
    page_content.append("")
    page_content.append("")

if USE_MARKDOWN:
    import markdown
    html = markdown.markdown(os.linesep.join(page_content), output_format="html5",
                            extensions=["markdown.extensions.extra"])
    print html
    import codecs
    output_file = codecs.open("combined_workplan.html", "w", encoding="utf-8", errors="xmlcharrefreplace")
    output_file.write(html)
else:
    print os.linesep.join(page_content)

This page provides the coordinated work plan between the Simulated OCS (SOCS) and the Scheduler.
Updated: 2015-07-31 13:23

h2. Combined Release v0.1
*Release Date:* 2015-07-31

|| SOCS Workplan || Scheduler Workplan ||
| * Release OpSim Requirements document
* Design SOCS independant of OpSim
* Define and initiate SOCS code modules in SE Simulations repository
* Prototype and benchmark Communications using current release of DDS-SAL-Python | * Release Scheduler Interface Document
* Design Scheduler independent from OpSim
* Define and initiate Scheduler code modules in T&S repository
* Prototype and benchmark Communications using current release of DDS-SAL-Python |


h2. Combined Release v0.2
*Release Date:* 2015-10-30

|| SOCS Workplan || Scheduler Workplan ||
| * Implement SOCS structure with interface
* Implement simulation kernel with time simulator
* Refactor and integrate observatory model
* Implement observatory telemetry simulator
* Implement simulation sequencer with targets h